In [1]:
import numpy as np
import first
import random
import typing

In [3]:
first.CyHexPosition(0, 0, 0).dist(first.CyHexPosition(-3, -2, 5))

5.0

In [7]:
def test_data(hmap, seed: int):
    random.seed(seed)
    all_positions = hmap.positions()
    avoidset = set(random.sample(all_positions, len(all_positions) // 4))
    start = random.choice(list(all_positions))
    end = start
    while end == start:
        end = random.choice(list(all_positions))
    return start, end, avoidset

def run_test(map_size: int, num_runs: int):
    hmap = first.HexMap(map_size)
    path_lengths = list()
    for i in range(num_runs):
        start, end, avoidset = test_data(hmap, i)
        path = start.shortest_path_dfs(end, avoidset, 2*map_size)
        path_lengths.append(len(path))
    return start, end, sum(pl for pl in path_lengths if pl is not None)

%timeit run_test(100, 100)

1.07 s ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
